# Business Problem

From https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

Can you predict which water pumps are faulty?

Using data from Taarifa and the Tanzanian Ministry of Water, can you predict which pumps are functional, which need some repairs, and which don't work at all? This is an intermediate-level practice competition. Predict one of these three classes based on a number of variables about what kind of pump is operating, when it was installed, and how it is managed. A smart understanding of which waterpoints will fail can improve maintenance operations and ensure that clean, potable water is available to communities across Tanzania.

In [6]:
# Imports

In [11]:
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from pandas_profiling import ProfileReport


# Obtain

In [8]:
# Obtained two csv files for target variable and features. Merged the dataframes. 

In [9]:
target_df = pd.read_csv("training_set_labels.csv")
features_df = pd.read_csv("training_set_values.csv")
df = pd.merge(target_df, features_df, on="id")
df

id    status_group  amount_tsh date_recorded           funder  \
0      69572      functional      6000.0     3/14/2011            Roman   
1       8776      functional         0.0      3/6/2013          Grumeti   
2      34310      functional        25.0     2/25/2013     Lottery Club   
3      67743  non functional         0.0     1/28/2013           Unicef   
4      19728      functional         0.0     7/13/2011      Action In A   
...      ...             ...         ...           ...              ...   
59395  60739      functional        10.0      5/3/2013  Germany Republi   
59396  27263      functional      4700.0      5/7/2011      Cefa-njombe   
59397  37057      functional         0.0     4/11/2011              NaN   
59398  31282      functional         0.0      3/8/2011            Malec   
59399  26348      functional         0.0     3/23/2011       World Bank   

       gps_height     installer  longitude   latitude              wpt_name  \
0            1390         Roman  34.938093  -9.856322                  none   
1            1399       GRUMETI  34.698766  -2.147466              Zahanati   
2             686  World vision  37.460664  -3.821329           Kwa Mahundi   
3             263        UNICEF  38.486161 -11.155298  Zahanati Ya Nanyumbu   
4               0       Artisan  31.130847  -1.825359               Shuleni   
...           ...           ...        ...        ...                   ...   
59395        1210           CES  37.169807  -3.253847   Area Three Namba 27   
59396        1212          Cefa  35.249991  -9.070629     Kwa Yahona Kuvala   
59397           0           NaN  34.017087  -8.750434               Mashine   
59398           0          Musa  35.861315  -6.378573                Mshoro   
59399         191         World  38.104048  -6.747464       Kwa Mzee Lugawa   

       ...  payment_type water_quality quality_group      quantity  \
0      ...      annually          soft          good        enough   
1      ...     never pay          soft          good  insufficient   
2      ...    per bucket          soft          good        enough   
3      ...     never pay          soft          good           dry   
4      ...     never pay          soft          good      seasonal   
...    ...           ...           ...           ...           ...   
59395  ...    per bucket          soft          good        enough   
59396  ...      annually          soft          good        enough   
59397  ...       monthly      fluoride      fluoride        enough   
59398  ...     never pay          soft          good  insufficient   
59399  ...    on failure         salty         salty        enough   

       quantity_group                source           source_type  \
0              enough                spring                spring   
1        insufficient  rainwater harvesting  rainwater harvesting   
2              enough                   dam                   dam   
3                 dry           machine dbh              borehole   
4            seasonal  rainwater harvesting  rainwater harvesting   
...               ...                   ...                   ...   
59395          enough                spring                spring   
59396          enough                 river            river/lake   
59397          enough           machine dbh              borehole   
59398    insufficient          shallow well          shallow well   
59399          enough          shallow well          shallow well   

      source_class              waterpoint_type waterpoint_type_group  
0      groundwater           communal standpipe    communal standpipe  
1          surface           communal standpipe    communal standpipe  
2          surface  communal standpipe multiple    communal standpipe  
3      groundwater  communal standpipe multiple    communal standpipe  
4          surface           communal standpipe    communal standpipe  
...            ...                          ...           

In [13]:
# profile = ProfileReport(df, title="Pandas Profiling Report")

In [14]:
# View dataframe statistics
# There are only 9 numerical columns. In light of this information, will make a vanilla model with only numerical 
# columns than add in categorical columns to see if that improves the results
df.describe()

id     amount_tsh    gps_height     longitude      latitude  \
count  59400.000000   59400.000000  59400.000000  59400.000000  5.940000e+04   
mean   37115.131768     317.650385    668.297239     34.077427 -5.706033e+00   
std    21453.128371    2997.574558    693.116350      6.567432  2.946019e+00   
min        0.000000       0.000000    -90.000000      0.000000 -1.164944e+01   
25%    18519.750000       0.000000      0.000000     33.090347 -8.540621e+00   
50%    37061.500000       0.000000    369.000000     34.908743 -5.021597e+00   
75%    55656.500000      20.000000   1319.250000     37.178387 -3.326156e+00   
max    74247.000000  350000.000000   2770.000000     40.345193 -2.000000e-08   

        num_private   region_code  district_code    population  \
count  59400.000000  59400.000000   59400.000000  59400.000000   
mean       0.474141     15.297003       5.629747    179.909983   
std       12.236230     17.587406       9.633649    471.482176   
min        0.000000      1.000000       0.000000      0.000000   
25%        0.000000      5.000000       2.000000      0.000000   
50%        0.000000     12.000000       3.000000     25.000000   
75%        0.000000     17.000000       5.000000    215.000000   
max     1776.000000     99.000000      80.000000  30500.000000   

       construction_year  
count       59400.000000  
mean         1300.652475  
std           951.620547  
min             0.000000  
25%             0.000000  
50%          1986.000000  
75%          2004.000000  
max          2013.000000

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   status_group           59400 non-null  object 
 2   amount_tsh             59400 non-null  float64
 3   date_recorded          59400 non-null  object 
 4   funder                 55765 non-null  object 
 5   gps_height             59400 non-null  int64  
 6   installer              55745 non-null  object 
 7   longitude              59400 non-null  float64
 8   latitude               59400 non-null  float64
 9   wpt_name               59400 non-null  object 
 10  num_private            59400 non-null  int64  
 11  basin                  59400 non-null  object 
 12  subvillage             59029 non-null  object 
 13  region                 59400 non-null  object 
 14  region_code            59400 non-null  int64  
 15  di

In [ ]:
df.shape

# Scrub

In [ ]:
# Drop unnecessary columns

In [ ]:
df.head()

In [ ]:
# check for missing values
df.isna().sum()

In [ ]:
df.isna().mean()

In [ ]:
# drop columns with missing values? 
# df.dropna(axis="columns").head()
# Or drop columns by arbitrary threshold
# ufo.dropna(thresh=len(ufo)*0.9, axis="columns").head()

In [ ]:
# Drop Id column, it is numerical, but does not give any viable information for modeling. 
#df.drop(["id"], axis=1, inplace=True)

In [ ]:
# remove duplicates from dataframe

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated().value_counts()

In [ ]:
duplicate = df[df.duplicated()]
 
print("Duplicate Rows :")
 
# Print the resultant Dataframe
duplicate

In [ ]:
# remove irrelevant columns

In [ ]:
df.columns

In [ ]:
# Removing columns that contain the same values as others
# Include a function for this?

In [ ]:
for col in df.columns:
    print(col)

In [ ]:
# "subvillage" vs "region" ?

In [ ]:
df_funct = df[df['status_group'] == 'functional']


In [ ]:
# Make this into a function

In [ ]:
res = df.groupby('region')['status_group'].value_counts(normalize=True)
res = pd.DataFrame(res)
res.columns = ['percent']
res.reset_index(inplace=True)
sns.boxplot(x = 'status_group', y = 'percent', data = res)

In [ ]:
res = df.groupby("region")["status_group"].value_counts(normalize=True)
res = pd.DataFrame(res)
res.columns = ['percent']
res.reset_index(inplace = True)
sns.boxplot(x = 'status_group', y = 'percent', data = res)

In [ ]:
df["subvillage"].nunique()

In [ ]:
df["region"].nunique()

In [ ]:
# "region_code" vs "district_code"

In [ ]:
len(df["region_code"].unique())

In [ ]:
len(df["district_code"].unique())

In [ ]:
# "lga" vs "ward"

In [ ]:
# "scheme_management" vs "scheme_name"

In [ ]:
len(df["scheme_management"].unique())

In [ ]:
len(df["scheme_name"].unique())

In [ ]:
df.drop("scheme_name", axis=1, inplace=True)

In [ ]:
# "extraction_type" vs "extraction_type_group" vs "extraction_type_class"
# Make graph

In [ ]:
len(df["extraction_type"].unique())

In [ ]:
len(df["extraction_type_group"].unique())

In [ ]:
len(df["extraction_type_class"].unique())

In [ ]:
df.drop(["extraction_type", "extraction_type_group"], axis=1, inplace=True)

In [ ]:
# "management" vs "management_group"
# Make a graph

In [ ]:
len(df["management"].unique())

In [ ]:
len(df["management_group"].unique())

In [ ]:
df.drop("management", axis=1, inplace=True)

In [ ]:
# "payment" vs "payment_type"
# Unique values ...

In [ ]:
len(df["payment"].unique())

In [ ]:
len(df["payment_type"].unique())

In [ ]:
df.drop(["payment_type"], axis=1, inplace=True)

In [ ]:
# "water_quality" vs "quality_group"

In [ ]:
df["water_quality"].unique()

In [ ]:
df["quality_group"].unique()

In [ ]:
# "quantity" vs "quantity_group"

In [ ]:
len(df["quantity"].unique())

In [ ]:
len(df["quantity_group"].unique())

In [ ]:
df.drop(["quantity_group"], axis=1, inplace=True)

In [ ]:
# "source" vs "source_type" vs "source_class"
len(df["source"].unique())

In [ ]:
df["source"].unique()

In [ ]:
df.shape

In [ ]:
df.drop(["source"], axis=1, inplace=True)

In [ ]:
len(df["source_type"].unique())

In [ ]:
df["source_type"].unique()

In [ ]:
len(df["source_class"].unique())

In [ ]:
df["source_class"].unique()

In [ ]:
df.drop(["source_class"], axis=1, inplace=True)

In [ ]:
# "waterpoint_type" vs "waterpoint_type_group"
len(df["waterpoint_type"].unique())

In [ ]:
len(df["waterpoint_type_group"].unique())

In [ ]:
# drop df["waterpoint_type"]
df.drop(["waterpoint_type"], axis=1, inplace=True)

In [ ]:
df.head()

# Explore

# Model 

In [ ]:
# Decision Tree

In [ ]:
X = df.select_dtypes("number").drop("id", axis=1)
y = df["status_group"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=23)

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

In [ ]:
dt.score(X_train, y_train)

In [ ]:
dt.score(X_test, y_test)

In [ ]:
# second vanilla model (accuracy score on test data is overfitting the test data)

In [ ]:
dt1 = DecisionTreeClassifier(max_depth=5)
dt1.fit(X_train, y_train)

In [ ]:
dt1.score(X_train, y_train)

In [ ]:
dt1.score(X_test, y_test)

# Interpret